# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [30]:
KEYWORDS = ['python', 'парсинг', 'java', 'россии']
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [3]:
posts = soup.find_all('article', class_='post')

In [31]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
my_df = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    texts = post.find_all('div', class_='post__text')
    for text_post in texts:
        text_lower = text_post.text.lower()
        if any([desired in text_lower for desired in KEYWORDS]):
            time = post.find('span', class_='post__time').text
            title_el = post.find('a', class_='post__title_link')
            link = title_el.attrs.get('href')
            row = {'date': time, 'title': title_el.text, 'link': link}
            my_df = pd.concat([my_df, pd.DataFrame([row])])
my_df

,date,title,link
0,сегодня в 12:21,"Как начать писать микросервис на Spring Boot, ...",https://habr.com/ru/post/517988/
0,сегодня в 12:20,"Картирование цифровых прав, часть IV. Право на...",https://habr.com/ru/post/517986/
0,вчера в 16:16,Сравнение Blazor и JavaScript-инструментов,https://habr.com/ru/company/ruvds/blog/517314/


### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [75]:
import time

In [87]:
EMAIL = ['fifa@gmail.com', 'master123@ya.ru']
URL = 'https://digibody.avast.com/v1/web/leaks'

In [131]:
my_df = pd.DataFrame()
for email_element in EMAIL: 
    time.sleep(0.3)
    res = requests.post(URL, json={'email':email_element})
    results_df = res.json()
    list_results = results_df['value']
    for item in list_results:
        row = {'email': item['username'], 'date': item['leak_info']['date'], "source": item['leak_info']['source_references'],'description': item['leak_info']['description']}
        my_df = pd.concat([my_df, pd.DataFrame([row])])
my_df

,email,date,source,description
0,fifa@gmail.com,1513900800000,[],The proliferation of stolen or leaked database...
0,fifa@gmail.com,1542758400000,[],"In July 2018, the online video site Animoto su..."
0,fifa@gmail.com,1535068800000,[],This combolist was compiled from a variety of ...
0,fifa@gmail.com,1549411200000,[],"On January 7, 2019, an online user named Sanix..."
0,fifa@gmail.com,1517616000000,[],"On an unconfirmed date, the manufacturing comp..."
...,...,...,...,...
0,fifa@gmail.com,1491782400000,[],This collection of compromised Gmail users con...
0,fifa@gmail.com,1575504000000,[],This source has been marked as sensitive due t...
0,fifa31,1477440000000,[],This source has been marked as sensitive due t...
0,zyka,1495756800000,[],"In February 2016, LEET's user database was all..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
